## 训练深度神经网络

### 梯度消失与梯度爆炸问题

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.models.Sequential([
    layers.Flatten(input_shape=[28, 28]),
    layers.BatchNormalization(),
    layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dense(10, activation='softmax'),
])